## Presentation Types with Hidden Types
## Also Network Visualization with Louvain Communities

* March 2022 version

    * Uses getDistance to identify `close matches` with side-by-side comparison of soggetti.  With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same.  These are reported as "flexed entries" in a separate column.

    * Labels Fuga, PEn, and ID according to time intervals.  
    * If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
    * Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
    * Counts number of entries
    * Provides offset and measure/beat locations
    * Sorts all presentation types by the order in which they appear in the piece
    * Reports voice names of the entries, in order of their appearance
    * Omits singleton soggetti (just one entry of a given motive in isolation)
    
    ALSO
    
    * Finds "hidden" types within a longer Fuga.  That is, if a 5-voice fuga also contains a PEN, it will label both of these as separate presentation type, along with all the relevant data noted above.

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


#### The following are special functions used by the classifier.  Don't change them.

In [2]:
def find_entry_int_distance(coordinates, piece: intervals.main_objs.ImportedPiece):
    
    """
    This function finds the melodic intervals between the first notes of 
    successive entries in a given presentation type.  
    They are represented as intervals with quality and direction, thus P-4, m3, P5, P5, M-9, P-4, P4
    
    """
    
    tone_list = []
    
    all_tones = piece.getNoteRest()
    
    for item in coordinates:
        filtered_tones = all_tones.loc[item] 
        tone_list.append(filtered_tones)
        
    noteObjects = [note.Note(tone) for tone in tone_list]
    _ints = [interval.Interval(noteObjects[i], noteObjects[i + 1]) for i in range(len(noteObjects) - 1)]
    entry_ints = []
    
    for _int in _ints:
        entry_ints.append(_int.directedName)
    
    return entry_ints

def split_by_threshold(seq, max_diff=70):  
    
    """
    This function finds gaps between sequences of matching melodic entries.  
    The threshold is set to 70 offsets by default--under about 10 measures.
    
    """
    it = iter(seq)
    last = next(it)
    part = [last]

    for curr in it:
        if curr - last > max_diff:
            yield part
            part = []

        part.append(curr)
        last = curr
#         print(part)
        
    yield part
    

def classify_by_offset(offset_diffs):
    """
    This function predicts the Presentation Types. It relies of the differences between 
    the first offsets of successive melodic entries. 
    
    If the offset differences are identical:  PEN
    If the odd-numbered offset differences are identical:  ID, since these represent
    situations in which the entries 1-2 have the same offset difference as entries 3-4
    If the offset differences are all different:  FUGA
    
    """
    alt_list = offset_diffs[::2]

    if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
        return 'PEN'
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
    elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
        return 'ID'
    elif len(offset_diffs) >= 1:
        return 'FUGA'

    

def temp_dict_of_details(slist, entry_array, det, matches):
    """
    This function assembles various features for the presentation types 
    into a single temporary dictionary, which in turn is appended to the dataframe of 'points'
    
    """
    
    array = entry_array[entry_array.index.get_level_values(0).isin(slist)]
    short_offset_list = array.index.to_list()
    time_ints = numpy.diff(array.index).tolist()
    voice_list = array['voice'].to_list()
    tone_coordinates =  list(zip(short_offset_list, voice_list))
    mel_ints = find_entry_int_distance(tone_coordinates, piece)
    first_offset = short_offset_list[0]
    meas_beat = det[det.index.get_level_values('Offset').isin(short_offset_list)]
    mb2 = meas_beat.reset_index()
    mb2['mb'] = mb2["Measure"].astype(str) + "/" + mb2["Beat"].astype(str)
    meas_beat_list = mb2['mb'].to_list()
    
    # temp results for this set
    temp = {"Composer": piece.metadata["composer"],
            "Title": piece.metadata["title"],
            'First_Offset': first_offset, 
            'Offsets': short_offset_list, 
            'Measures_Beats': meas_beat_list,
            "Soggetti": matches,
            'Voices': voice_list, 
            'Time_Entry_Intervals': time_ints, 
            'Melodic_Entry_Intervals': mel_ints}
    return temp

def classify_entries_as_presentation_types(piece, dur_ng, mel_ng, include_hidden_types):
    
    """
    This function uses several other functions to classify the entries in a given piece.
    The output is a list, in order of offset, of each presentation type, including information about
    measures/beats
    starting offset
    soggetti involved 
    melodic intervals of entry
    time intervals of entry
    
    set the length of the soggetti with `melodic_ngram_length`
    set the maximum difference between similar soggetti with `edit_distance_threshold`
    for chromatic vs diatonic, compound, and directed data in soggetti, see `interval_settings`
    to include all the hidden PENs and IDS (those found within longer Fugas), 
    use `include_hidden_types == True`.  
    For faster (and simpler) listing of points of imitation without hidden forms, use `include_hidden_types == False`
    
    
    """
    # Classifier with Functions
    points = pd.DataFrame()
    points2 = pd.DataFrame()
    # new_offset_list = []
#     nr = piece.getNoteRest()
    det = piece.detailIndex(nr, offset=True)

    # The following are now all set in the Notebook as argument
    # durations and ngrams of durations
    # dur = piece.getDuration(df=nr)
    # dur_ng = piece.getNgrams(df=dur, n=3)

    # ngrams of melodic entries
    # for chromatic, use:
    # piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
    #mel_ng = piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
    mels_stacked = mel_ng.stack().to_frame()
    mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

    # edit distance, based on side-by-side comparison of melodic ngrams
    # gets flexed and other similar soggetti
    dist = piece.getDistance(mel_ng)
    dist_stack = dist.stack().to_frame()


    # filter distances to threshold.  <2 is good
    distance_factor = edit_distance_threshold + 1
    filtered_dist_stack = dist_stack[dist_stack[0] < distance_factor]
    filtered_dist = filtered_dist_stack.reset_index()
    filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

    # Group the filtered distanced patterns
    full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

    if include_hidden_types == False:

        for matches in full_list_of_matches["match"]:
            related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
            entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
            offset_list = entry_array.index.to_list()
            split_list = list(split_by_threshold(offset_list))
            # here is the list of starting offsets of the original set of entries:  slist
            slist = split_list[0]
            temp = temp_dict_of_details(slist, entry_array, det, matches)

            points = points.append(temp, ignore_index=True)
            points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
            points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
            points = points[points['Offsets'].apply(len) > 1]

        points["Offsets_Key"] = points["Offsets"].apply(offset_joiner)
        points.drop_duplicates(subset=["Offsets_Key"], keep='first', inplace = True)
        points['Flexed_Entries'] = points["Soggetti"].apply(len) > 1
        points["Number_Entries"] = points["Offsets"].apply(len) 
        col_order = ['Composer', 
                 'Title', 
                 'First_Offset', 
                 'Measures_Beats', 
                 'Melodic_Entry_Intervals',
                 'Offsets', 
                 'Soggetti', 
                 'Time_Entry_Intervals', 
                 'Voices',
                 'Presentation_Type', 
                  'Number_Entries',
                'Flexed_Entries']
        points = points.reindex(columns=col_order)
        return points
    
    elif include_hidden_types == True:
        
        for matches in full_list_of_matches["match"]:
            related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
            entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
            offset_list = entry_array.index.to_list()
            split_list = list(split_by_threshold(offset_list))
            # here is the list of starting offsets of the original set of entries:  slist
            slist = split_list[0]
            temp = temp_dict_of_details(slist, entry_array, det, matches)

            points = points.append(temp, ignore_index=True)
            points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
            points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
            points = points[points['Offsets'].apply(len) > 1]
            
#   here is where we check for all combinations of types within the longer patterns        


            l = len(slist)
            m = l + 1
            if l > 2:
                for r in range(3, m):
                    list_combinations = list(combinations(slist, r))
                    for tiny_list in list_combinations:

                        temp = temp_dict_of_details(tiny_list, entry_array, det, matches)

                        temp["Presentation_Type"] = classify_by_offset(temp['Time_Entry_Intervals'])

                        if 'PEN' in temp["Presentation_Type"]:
                            points2 = points2.append(temp, ignore_index=True)#.sort_values("First_Offset")

                        if 'ID' in temp["Presentation_Type"]:
                            points2 = points2.append(temp, ignore_index=True)#.sort_values("First_Offset"



        points_combined = points.append(points2, ignore_index=True).sort_values("First_Offset").reset_index(drop=True)
        points_combined["Offsets_Key"] = points_combined["Offsets"].apply(offset_joiner)
        points_combined.drop_duplicates(subset=["Offsets_Key"], keep='first', inplace = True)
        points_combined['Flexed_Entries'] = points_combined["Soggetti"].apply(len) > 1
        points_combined["Number_Entries"] = points_combined["Offsets"].apply(len) 
        col_order = ['Composer', 
                 'Title', 
                 'First_Offset', 
                 'Measures_Beats', 
                 'Melodic_Entry_Intervals',
                 'Offsets', 
                 'Soggetti', 
                 'Time_Entry_Intervals', 
                 'Voices',
                 'Presentation_Type', 
                  'Number_Entries',
                'Flexed_Entries']
        points_combined = points_combined.reindex(columns=col_order).reset_index()
        return points_combined


# #  the following are used to turn the offset diffs and melodic entry intervals 
# # and melodies into strings for the network 


def joiner(a):
    b = '_'.join(map(str, a))
    return b
def clean_melody(c):  
    d = (('_').join(c[0].split(","))).replace(" ", "")
    return d
def offset_joiner(a):
    b = '_'.join(map(str, a))
    return b

## Load one Piece Here

* Note that you can load from CRIM, or put a file in the **Music_Files** folder in the Notebook.

In [3]:
# piece = importScore('Music_Files/Senfl_Ave_forCRIM.mei_msg.mei')
piece = importScore('https://crimproject.org/mei/CRIM_Model_0019.mei')
# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')


Successfully imported https://crimproject.org/mei/CRIM_Model_0019.mei


## Run the Classifier Here

- set the length of the soggetti with `melodic_ngram_length`
- set the maximum difference between similar soggetti with `edit_distance_threshold`
- for chromatic vs diatonic, compound, and directed data in soggetti, see `interval_settings`
- to include all the hidden PENs and IDS (those found within longer Fugas, use `include_hidden_types == True`.  
- for faster (and simpler) listing of points of imitation without hidden forms, use `include_hidden_types == False`



In [4]:
include_hidden_types = False
melodic_ngram_length = 4
edit_distance_threshold = 1
nr = piece.getNoteRest()
dur = piece.getDuration(df=nr)
dur_ng = piece.getNgrams(df=dur, n=melodic_ngram_length)
mel_ng = piece.getMelodicEntries(interval_settings=('d', True, True), n=melodic_ngram_length)
output = classify_entries_as_presentation_types(piece, dur_ng, mel_ng, include_hidden_types)
# len(output)

In [5]:
output

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4.0, 59/1....","[P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[394.0, 406.0, 418.0, 434.0, 446.0, 464.0, 486...","[-2, 2, 1, 3]","[12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0, 16.0]","[Altus, Cantus, Bassus, Tenor, Altus, Bassus, ...",FUGA,9,False
1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8, P1, P8, P-5, P-4,...","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0, 150...","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0, 22.0, 16.0,...","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,15,True
2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 40/2.0, 42/2.0, 44/1.0, 46/2....","[P-5, P8, P-12, P8, P8, P-11]","[298.0, 302.0, 314.0, 330.0, 344.0, 362.0, 382.0]","[1, 1, -3, 3]","[4.0, 12.0, 16.0, 14.0, 18.0, 20.0]","[Altus, Tenor, Cantus, Bassus, Altus, Cantus, ...",FUGA,7,False


### Run Classifier on Several Pieces at Once

In [ ]:
piece_list = ['https://crimproject.org/mei/CRIM_Model_0008.mei',
             'https://crimproject.org/mei/CRIM_Mass_0005_1.mei',
             'https://crimproject.org/mei/CRIM_Mass_0005_2.mei',
             'https://crimproject.org/mei/CRIM_Mass_0005_3.mei',
             'https://crimproject.org/mei/CRIM_Mass_0005_4.mei',
             'https://crimproject.org/mei/CRIM_Mass_0005_5.mei',
             'https://crimproject.org/mei/CRIM_Model_0001.mei',
             'https://crimproject.org/mei/CRIM_Mass_0002_1.mei',
             'https://crimproject.org/mei/CRIM_Mass_0002_2.mei',
             'https://crimproject.org/mei/CRIM_Mass_0002_3.mei',
             'https://crimproject.org/mei/CRIM_Mass_0002_4.mei',
             'https://crimproject.org/mei/CRIM_Mass_0002_5.mei',
             'https://crimproject.org/mei/CRIM_Model_0015.mei',
             'https://crimproject.org/mei/CRIM_Mass_0013_1.mei',
             'https://crimproject.org/mei/CRIM_Mass_0013_2.mei',
             'https://crimproject.org/mei/CRIM_Mass_0013_3.mei',
             'https://crimproject.org/mei/CRIM_Mass_0013_4.mei',
             'https://crimproject.org/mei/CRIM_Mass_0013_5.mei',
             'https://crimproject.org/mei/CRIM_Model_0019.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_1.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_2.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_3.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_4.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_5.mei']



In [3]:
piece_list = ['https://crimproject.org/mei/CRIM_Model_0019.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_1.mei']

In [9]:
include_hidden_types = True
melodic_ngram_length = 4
edit_distance_threshold = 1
final = pd.DataFrame()
for work in piece_list:
#     print(work)
    piece = importScore(work)   
    nr = piece.getNoteRest()

    dur = piece.getDuration(df=nr)
    dur_ng = piece.getNgrams(df=dur, n=melodic_ngram_length)
    mel_ng = piece.getMelodicEntries(interval_settings=('d', True, True), n=melodic_ngram_length)
    output = classify_entries_as_presentation_types(piece, dur_ng, mel_ng, include_hidden_types)
#     print(output)
    final = final.append(output, ignore_index=True)
final

Memoized piece detected.
Memoized piece detected.


,index,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4,True
1,1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 11/3.0, 21/4.0, 32/2.0]","[P-5, P5, P4]","[0.0, 84.0, 166.0, 250.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[84.0, 82.0, 84.0]","[Cantus, Altus, Cantus, Cantus]",ID,4,True
2,2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 7/1.0, 21/4.0, 27/4.0]","[P-12, P12, P-12]","[0.0, 48.0, 166.0, 214.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[48.0, 118.0, 48.0]","[Cantus, Bassus, Cantus, Bassus]",ID,4,True
3,3,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 7/1.0, 19/4.0, 25/4.0]","[P-12, P5, P1]","[0.0, 48.0, 150.0, 198.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[48.0, 102.0, 48.0]","[Cantus, Bassus, Tenor, Tenor]",ID,4,True
4,4,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 6/1.0, 22/4.0, 27/4.0]","[P-8, P4, P-8]","[0.0, 40.0, 174.0, 214.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[40.0, 134.0, 40.0]","[Cantus, Tenor, Altus, Bassus]",ID,4,True
5,5,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 21/4.0, 22/4.0]","[P-5, P5, P-5]","[0.0, 8.0, 166.0, 174.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 158.0, 8.0]","[Cantus, Altus, Cantus, Altus]",ID,4,True
6,9,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 21/4.0, 22/4.0]","[P-5, P-4, P-5, P12, P-5]","[0.0, 8.0, 40.0, 48.0, 166.0, 174.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0, 118.0, 8.0]","[Cantus, Altus, Tenor, Bassus, Cantus, Altus]",ID,6,True
7,13,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8, P1, P8, P-5, P-4,...","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0, 150...","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0, 22.0, 16.0,...","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,15,True
8,15,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,8.0,"[2/1.0, 6/1.0, 21/4.0, 25/4.0]","[P-4, P8, P-8]","[8.0, 40.0, 166.0, 198.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[32.0, 126.0, 32.0]","[Altus, Tenor, Cantus, Tenor]",ID,4,True
9,16,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,8.0,"[2/1.0, 6/1.0, 25/4.0, 29/4.0]","[P-4, P1, P4]","[8.0, 40.0, 198.0, 230.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[32.0, 158.0, 32.0]","[Altus, Tenor, Tenor, Altus]",ID,4,True


In [ ]:
output

In [10]:
final["MINT"] = final["Melodic_Entry_Intervals"].apply(joiner)
final["TINT"] = final["Time_Entry_Intervals"].apply(joiner)

final['SOG'] = final['Soggetti'].apply(clean_melody)
final['ALL'] = final["MINT"] + '_' + final["TINT"] + '_' + final['SOG']

final["ALL_INT"] = final["MINT"] + '_' + final["TINT"]
final["ALL_SOG"] = final["MINT"] + '_' + final["SOG"]
final

,index,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,MINT,TINT,SOG,ALL,ALL_INT,ALL_SOG
0,0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4,True,P-5_P-4_P-5,8.0_32.0_8.0,-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0_-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0,P-5_P-4_P-5_-3_3_2_-2
1,1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 11/3.0, 21/4.0, 32/2.0]","[P-5, P5, P4]","[0.0, 84.0, 166.0, 250.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[84.0, 82.0, 84.0]","[Cantus, Altus, Cantus, Cantus]",ID,4,True,P-5_P5_P4,84.0_82.0_84.0,-3_3_2_-2,P-5_P5_P4_84.0_82.0_84.0_-3_3_2_-2,P-5_P5_P4_84.0_82.0_84.0,P-5_P5_P4_-3_3_2_-2
2,2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 7/1.0, 21/4.0, 27/4.0]","[P-12, P12, P-12]","[0.0, 48.0, 166.0, 214.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[48.0, 118.0, 48.0]","[Cantus, Bassus, Cantus, Bassus]",ID,4,True,P-12_P12_P-12,48.0_118.0_48.0,-3_3_2_-2,P-12_P12_P-12_48.0_118.0_48.0_-3_3_2_-2,P-12_P12_P-12_48.0_118.0_48.0,P-12_P12_P-12_-3_3_2_-2
3,3,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 7/1.0, 19/4.0, 25/4.0]","[P-12, P5, P1]","[0.0, 48.0, 150.0, 198.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[48.0, 102.0, 48.0]","[Cantus, Bassus, Tenor, Tenor]",ID,4,True,P-12_P5_P1,48.0_102.0_48.0,-3_3_2_-2,P-12_P5_P1_48.0_102.0_48.0_-3_3_2_-2,P-12_P5_P1_48.0_102.0_48.0,P-12_P5_P1_-3_3_2_-2
4,4,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 6/1.0, 22/4.0, 27/4.0]","[P-8, P4, P-8]","[0.0, 40.0, 174.0, 214.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[40.0, 134.0, 40.0]","[Cantus, Tenor, Altus, Bassus]",ID,4,True,P-8_P4_P-8,40.0_134.0_40.0,-3_3_2_-2,P-8_P4_P-8_40.0_134.0_40.0_-3_3_2_-2,P-8_P4_P-8_40.0_134.0_40.0,P-8_P4_P-8_-3_3_2_-2
5,5,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 21/4.0, 22/4.0]","[P-5, P5, P-5]","[0.0, 8.0, 166.0, 174.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 158.0, 8.0]","[Cantus, Altus, Cantus, Altus]",ID,4,True,P-5_P5_P-5,8.0_158.0_8.0,-3_3_2_-2,P-5_P5_P-5_8.0_158.0_8.0_-3_3_2_-2,P-5_P5_P-5_8.0_158.0_8.0,P-5_P5_P-5_-3_3_2_-2
6,9,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 21/4.0, 22/4.0]","[P-5, P-4, P-5, P12, P-5]","[0.0, 8.0, 40.0, 48.0, 166.0, 174.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0, 118.0, 8.0]","[Cantus, Altus, Tenor, Bassus, Cantus, Altus]",ID,6,True,P-5_P-4_P-5_P12_P-5,8.0_32.0_8.0_118.0_8.0,-3_3_2_-2,P-5_P-4_P-5_P12_P-5_8.0_32.0_8.0_118.0_8.0_-3_...,P-5_P-4_P-5_P12_P-5_8.0_32.0_8.0_118.0_8.0,P-5_P-4_P-5_P12_P-5_-3_3_2_-2
7,13,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8, P1, P8, P-5, P-4,...","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0, 150...","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0, 22.0, 16.0,...","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,15,True,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,8.0_32.0_8.0_36.0_20.0_24.0_22.0_16.0_8.0_24.0...,-3_3_2_-2,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...
8,15,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,8.0,"[2/1.0, 6/1.0, 21/4.0, 25/4.0]","[P-4, P8, P-8]","[8.0, 40.0, 166.0, 198.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[32.0, 126.0, 32.0]","[Altus, Tenor, Cantus, Tenor]",ID,4,True,P-4_P8_P-8,32.0_126.0_32.0,-3_3_2_-2,P-4_P8_P-8_32.0_126.0_32.0_-3_3_2_-2,P-4_P8_P-8_32.0_126.0_32.0,P-4_P8_P-8_-3_3_2_-2
9,16,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,8.0,"[2/1.0, 6/1.0, 25/4.0, 29/4.0]","[P-4, P1, P4]","[8.0, 40.0, 198.0, 230.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[32.0, 158.0,

In [12]:
filtered = final.loc[final['Number_Entries'] < 20] 
filtered

,index,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,MINT,TINT,SOG,ALL,ALL_INT,ALL_SOG
0,0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4,True,P-5_P-4_P-5,8.0_32.0_8.0,-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0_-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0,P-5_P-4_P-5_-3_3_2_-2
1,1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 11/3.0, 21/4.0, 32/2.0]","[P-5, P5, P4]","[0.0, 84.0, 166.0, 250.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[84.0, 82.0, 84.0]","[Cantus, Altus, Cantus, Cantus]",ID,4,True,P-5_P5_P4,84.0_82.0_84.0,-3_3_2_-2,P-5_P5_P4_84.0_82.0_84.0_-3_3_2_-2,P-5_P5_P4_84.0_82.0_84.0,P-5_P5_P4_-3_3_2_-2
2,2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 7/1.0, 21/4.0, 27/4.0]","[P-12, P12, P-12]","[0.0, 48.0, 166.0, 214.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[48.0, 118.0, 48.0]","[Cantus, Bassus, Cantus, Bassus]",ID,4,True,P-12_P12_P-12,48.0_118.0_48.0,-3_3_2_-2,P-12_P12_P-12_48.0_118.0_48.0_-3_3_2_-2,P-12_P12_P-12_48.0_118.0_48.0,P-12_P12_P-12_-3_3_2_-2
3,3,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 7/1.0, 19/4.0, 25/4.0]","[P-12, P5, P1]","[0.0, 48.0, 150.0, 198.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[48.0, 102.0, 48.0]","[Cantus, Bassus, Tenor, Tenor]",ID,4,True,P-12_P5_P1,48.0_102.0_48.0,-3_3_2_-2,P-12_P5_P1_48.0_102.0_48.0_-3_3_2_-2,P-12_P5_P1_48.0_102.0_48.0,P-12_P5_P1_-3_3_2_-2
4,4,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 6/1.0, 22/4.0, 27/4.0]","[P-8, P4, P-8]","[0.0, 40.0, 174.0, 214.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[40.0, 134.0, 40.0]","[Cantus, Tenor, Altus, Bassus]",ID,4,True,P-8_P4_P-8,40.0_134.0_40.0,-3_3_2_-2,P-8_P4_P-8_40.0_134.0_40.0_-3_3_2_-2,P-8_P4_P-8_40.0_134.0_40.0,P-8_P4_P-8_-3_3_2_-2
5,5,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 21/4.0, 22/4.0]","[P-5, P5, P-5]","[0.0, 8.0, 166.0, 174.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 158.0, 8.0]","[Cantus, Altus, Cantus, Altus]",ID,4,True,P-5_P5_P-5,8.0_158.0_8.0,-3_3_2_-2,P-5_P5_P-5_8.0_158.0_8.0_-3_3_2_-2,P-5_P5_P-5_8.0_158.0_8.0,P-5_P5_P-5_-3_3_2_-2
6,9,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 21/4.0, 22/4.0]","[P-5, P-4, P-5, P12, P-5]","[0.0, 8.0, 40.0, 48.0, 166.0, 174.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0, 118.0, 8.0]","[Cantus, Altus, Tenor, Bassus, Cantus, Altus]",ID,6,True,P-5_P-4_P-5_P12_P-5,8.0_32.0_8.0_118.0_8.0,-3_3_2_-2,P-5_P-4_P-5_P12_P-5_8.0_32.0_8.0_118.0_8.0_-3_...,P-5_P-4_P-5_P12_P-5_8.0_32.0_8.0_118.0_8.0,P-5_P-4_P-5_P12_P-5_-3_3_2_-2
7,13,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8, P1, P8, P-5, P-4,...","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0, 150...","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0, 22.0, 16.0,...","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,15,True,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,8.0_32.0_8.0_36.0_20.0_24.0_22.0_16.0_8.0_24.0...,-3_3_2_-2,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...
8,15,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,8.0,"[2/1.0, 6/1.0, 21/4.0, 25/4.0]","[P-4, P8, P-8]","[8.0, 40.0, 166.0, 198.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[32.0, 126.0, 32.0]","[Altus, Tenor, Cantus, Tenor]",ID,4,True,P-4_P8_P-8,32.0_126.0_32.0,-3_3_2_-2,P-4_P8_P-8_32.0_126.0_32.0_-3_3_2_-2,P-4_P8_P-8_32.0_126.0_32.0,P-4_P8_P-8_-3_3_2_-2
9,16,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,8.0,"[2/1.0, 6/1.0, 25/4.0, 29/4.0]","[P-4, P1, P4]","[8.0, 40.0, 198.0, 230.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[32.0, 158.0,

### Network Visualization with Louvain Communities

In [13]:

f2 = filtered.groupby('MINT')['Title'].apply(list).reset_index()
f2


,MINT,Title
0,M-9_P15,[Veni sponsa Christi]
1,P-12_P12_P-12,[Veni sponsa Christi]
2,P-12_P5_P1,[Veni sponsa Christi]
3,P-4_P-5_P-4,[Missa Veni sponsa Christi: Kyrie]
4,P-4_P11_P-11,[Veni sponsa Christi]
5,P-4_P1_P4,[Veni sponsa Christi]
6,P-4_P4_P-4,[Veni sponsa Christi]
7,P-4_P8_P-8,[Veni sponsa Christi]
8,P-5_P-4_P-5,"[Veni sponsa Christi, Missa Veni sponsa Christ..."
9,P-5_P-4_P-5_P12_P-5,[Veni sponsa Christi]


In [14]:
pairs = f2.Title.apply(lambda x: list(combinations(x, 2)))
pairs

0                                                    []
1                                                    []
2                                                    []
3                                                    []
4                                                    []
5                                                    []
6                                                    []
7                                                    []
8     [(Veni sponsa Christi, Missa Veni sponsa Chris...
9                                                    []
10                                                   []
11                                                   []
12                                                   []
13                                                   []
14                                                   []
15                                                   []
16                                                   []
17                                              

In [15]:
pairs2 = pairs.explode().dropna()
pairs2

8     (Veni sponsa Christi, Missa Veni sponsa Christ...
38           (Veni sponsa Christi, Veni sponsa Christi)
49           (Veni sponsa Christi, Veni sponsa Christi)
Name: Title, dtype: object

In [16]:
unique_pairs = pairs.explode().dropna().unique()
unique_pairs

array([('Veni sponsa Christi', 'Missa Veni sponsa Christi: Kyrie'),
       ('Veni sponsa Christi', 'Veni sponsa Christi')], dtype=object)

In [17]:
pd.Series(unique_pairs).isna().sum()

0

In [18]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G

In [19]:
pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="white", font_color="black")

In [20]:
G = nx.Graph()
G.add_edges_from(unique_pairs)
G = add_communities(G)

In [21]:
pyvis_graph.from_nx(G)

In [22]:
pyvis_graph.show('MINT.html')


#### Below is Development Work

In [ ]:
filtered = output.loc[output['Number_Entries'] < 4] 
filtered

In [ ]:
output = output.loc[output['Presentation_Type'] == "PEN"] 
output

In [ ]:
offset_diffs = [2.0, 1.0, 2.0, 3.0, 5.0, 6.0]
# some_list[start:stop:step]
alt_list = offset_diffs[::2]
alt_list

In [ ]:
# this works with ONE list of offsets

points2 = pd.DataFrame()
split_list = [90.0, 94.0, 102.0, 106.0, 134.0, 146.0, 162.0]

l = len(split_list)  
for r in range(3, l):
    list_combinations = list(combinations(split_list, r))
#             combo_time_ints = []
    for combo in list_combinations:
        combo_time_ints = numpy.diff(combo).tolist()
        combo_array = entry_array[entry_array.index.get_level_values(0).isin(combo)]
        combo_voice_list = combo_array['voice'].to_list()
        combo_patterns = combo_array['pattern']
        unique_combo_patterns = list(set(combo_patterns))
        tone_coordinates =  list(zip(combo, combo_voice_list))
# tone_coordinates.ffill(inplace=True)
        mel_ints = find_entry_int_distance(tone_coordinates, piece)
        hidden_type = classify_by_offset(combo_time_ints)

        meas_beat = det[det.index.get_level_values('Offset').isin(combo)]
        mb2 = meas_beat.reset_index()
        mb2['mb'] = mb2["Measure"].astype(str) + "/" + mb2["Beat"].astype(str)
        meas_beat_list = mb2['mb'].to_list()

        combo_temp = {'First_Offset': combo[0], 
            'Offsets': combo, 
            'Measures_Beats': meas_beat_list,
            'Presentation_Type': hidden_type,
            "Soggetti": unique_combo_patterns,
            'Voices': combo_voice_list, 
            'Time_Entry_Intervals': combo_time_ints, 
            'Melodic_Entry_Intervals': mel_ints}

        if 'PEN' in hidden_type:
            points2 = points2.append(combo_temp, ignore_index=True).sort_values("First_Offset")
#             points2 = points2[points2['Offsets'].apply(len) > 1]
        if 'ID' in hidden_type:
            points2 = points2.append(combo_temp, ignore_index=True).sort_values("First_Offset")
#             points2 = points2[points2['Offsets'].apply(len) > 1]
        
        
# combo_time_ints
# combo_array
# # combo_voice_list
# # combo_patterns
# # unique_combo_patterns
# # tone_coordinates
# # mel_ints
# # combo_temp
points2

In [ ]:
# this finds hidden fugas.  
# try to run each of the first set of results above ('points') through this tool, then append the 
# new results to the full DF, and sort again.  
# mark each long pattern with 'has hidden pattern' boolean?  or ?

sample_list = points["Offsets"][4]

hidden_pts = []
n = len(sample_list)
for item in range(3, n):
    list_combinations = list(combinations(sample_list, item))
    for group in list_combinations:
        group_time_ints = numpy.diff(group).tolist()
        hidden_type = classify_by_offset(group_time_ints)
        if 'PEN' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
        if 'ID' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
        

list_combinations

In [ ]:
def classify_entries_as_presentation_types(piece):
    # Classifier with Functions
    points = pd.DataFrame()
    points2 = pd.DataFrame()
    # new_offset_list = []
    nr = piece.getNoteRest()
    det = piece.detailIndex(nr, offset=True)

    # durations and ngrams of durations
    dur = piece.getDuration(df=nr)
    dur_ng = piece.getNgrams(df=dur, n=4)

    # ngrams of melodic entries
    # for chromatic, use:
    # piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
    mel = piece.getMelodicEntries(n=4)
    mels_stacked = mel.stack().to_frame()
    mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

    # edit distance, based on side-by-side comparison of melodic ngrams
    # gets flexed and other similar soggetti
    dist = piece.getDistance(mel)
    dist_stack = dist.stack().to_frame()


    # filter distances to threshold.  <2 is good
    filtered_dist_stack = dist_stack[dist_stack[0] < 2]
    filtered_dist = filtered_dist_stack.reset_index()
    filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

    # Group the filtered distanced patterns
    full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

    for matches in full_list_of_matches["match"]:
        related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
        entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
        offset_list = entry_array.index.to_list()
        split_list = list(split_by_threshold(offset_list))
        # here is the list of starting offsets of the original set of entries:  slist
        slist = split_list[0]
        temp = temp_dict_of_details(slist, entry_array, det, matches)

        points = points.append(temp, ignore_index=True)
        points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
        points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
        points = points[points['Offsets'].apply(len) > 1]

        l = len(slist)
        if l > 2:
            for r in range(3, l):
    #             list_combinations = list(combinations(slist, r))
                list_combinations = list(combinations(slist, r))
                for slist in list_combinations:

                    temp = temp_dict_of_details(slist, entry_array, det, matches)

                    temp["Presentation_Type"] = classify_by_offset(temp['Time_Entry_Intervals'])

                    if 'PEN' in temp["Presentation_Type"]:
                        points2 = points2.append(temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
                        points2 = points2[points2['Offsets'].apply(len) > 1]
                    if 'ID' in temp["Presentation_Type"]:
                        points2 = points2.append(combo_temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
                points2.sort_values("First_Offset")
                points2.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)

    points_combined = points.append(points2, ignore_index=True).sort_values("First_Offset").reset_index(drop=True)
    points_combined['Flexed_Entries'] = points_combined["Soggetti"].apply(len) > 1
    points_combined["Number_Entries"] = points["Offsets"].apply(len)     
    return points2


In [37]:
# This test works

points = pd.DataFrame()
points2 = pd.DataFrame()
# new_offset_list = []
nr = piece.getNoteRest()
det = piece.detailIndex(nr, offset=True)

# durations and ngrams of durations
dur = piece.getDuration(df=nr)
dur_ng = piece.getNgrams(df=dur, n=4)

# ngrams of melodic entries
# for chromatic, use:
# piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
mel = piece.getMelodicEntries(n=4)
mels_stacked = mel.stack().to_frame()
mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

# edit distance, based on side-by-side comparison of melodic ngrams
# gets flexed and other similar soggetti
dist = piece.getDistance(mel)
dist_stack = dist.stack().to_frame()


# filter distances to threshold.  <2 is good
filtered_dist_stack = dist_stack[dist_stack[0] < 2]
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

# Group the filtered distanced patterns
full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

for matches in full_list_of_matches["match"]:
    related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
    entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
    offset_list = entry_array.index.to_list()
    split_list = list(split_by_threshold(offset_list))
    # here is the list of starting offsets of the original set of entries:  slist
    slist = split_list[0]
    temp = temp_dict_of_details(slist, entry_array, det, matches)

    points = points.append(temp, ignore_index=True)
    points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
    points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
    points = points[points['Offsets'].apply(len) > 1]

    test = [278.0, 286.0, 294.0, 298.0, 306.0, 310.0]

    l = len(test)  
    for item in range(3, l):
        list_combinations = list(combinations(test, item))
        for group in list_combinations:
            group_time_ints = numpy.diff(group).tolist()
            hidden_type = classify_by_offset(group_time_ints)
            for item in group:
    #         print(item)
                array = group[entry_array.index.get_level_values(0).isin(item)]
                short_offset_list = array.index.to_list()
                time_ints = numpy.diff(array.index).tolist()
                voice_list = array['voice'].to_list()
                if 'PEN' in hidden_type:
                    print(group)
                    print(group_time_ints)
                    print(hidden_type)
                    hidden_pts.append(group_time_ints)
                if 'ID' in hidden_type:
                    print(group)
                    print(group_time_ints)
                    print(hidden_type)
                    hidden_pts.append(group_time_ints)
# len(split_list[0])           

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [float]

In [ ]:
#  This shows how the classifier works:

if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
    print('This is a PEN')
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
    print('This is an ID')
elif len(offset_diffs) >= 1:
    print('This is a FUGA')

In [ ]:
# This shows how combinations works for a given set of time intervals
offset_diffs = [12.0, 32.0, 12.0, 4.0]
l = len(offset_diffs)
# print(l)
if l > 2:
    for r in range(3, l):
        print(r)
        list_combinations = list(combinations(offset_diffs, r))
#         for slist in list_combinations:
        print(list_combinations)

In [28]:
slist = [278.0, 286.0, 294.0, 298.0, 306.0, 310.0]
l = len(slist)
# for r in range(3, 6):
list_combinations = list(combinations(slist, 4))
#     for tiny_list in list_combinations:

In [29]:
print(list_combinations)

[(278.0, 286.0, 294.0, 298.0), (278.0, 286.0, 294.0, 306.0), (278.0, 286.0, 294.0, 310.0), (278.0, 286.0, 298.0, 306.0), (278.0, 286.0, 298.0, 310.0), (278.0, 286.0, 306.0, 310.0), (278.0, 294.0, 298.0, 306.0), (278.0, 294.0, 298.0, 310.0), (278.0, 294.0, 306.0, 310.0), (278.0, 298.0, 306.0, 310.0), (286.0, 294.0, 298.0, 306.0), (286.0, 294.0, 298.0, 310.0), (286.0, 294.0, 306.0, 310.0), (286.0, 298.0, 306.0, 310.0), (294.0, 298.0, 306.0, 310.0)]


In [30]:
list_offsets = [294.0, 298.0, 306.0, 310.0]

In [23]:
offset_diffs = [4, 5, 6]

In [24]:
alt_list = offset_diffs[::2]

if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
    print('This is a PEN')
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
    print('This is an ID')
elif len(offset_diffs) >= 1:
    print('This is a FUGA')

This is a FUGA
